#Libraries

In [3]:
import numpy as np
from scipy.io import wavfile
import os
import csv

#Settings

In [4]:
sr = 44100
out_dir = "maqam_53tet"
os.makedirs(out_dir, exist_ok=True)
REF_FREQ = 440.0 
REF_INDEX = 0

#Functions

In [5]:
def get_freq(comma_index):
    return REF_FREQ * (2 ** (comma_index / 53.0))
    

In [6]:
def generate_tone(f, duration, sr):
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)
    
    audio = 0.5 * np.sin(2 * np.pi * f * t) + \
            0.3 * np.sin(2 * np.pi * (2*f) * t) + \
            0.15 * np.sin(2 * np.pi * (3*f) * t) + \
            0.05 * np.sin(2 * np.pi * (4*f) * t)
    
    
    envelope = np.ones_like(t)
    attack = int(0.05 * sr)
    release = int(0.1 * sr)
    envelope[:attack] = np.linspace(0, 1, attack)
    envelope[-release:] = np.linspace(1, 0, release)
    
    return audio * envelope

#  MAQAM DEFINITIONS (Based on Dügah(A)) 
Intervals (Comma): T=9, K=8, S=5, B=4, A=12 or 13
Using comma_offsets list for naming convenience.
0 = A (Dügah)

In [8]:
maqam_data = {
    "Buselik_A": {
        # The maqam closest to the Natural Minor scale in Western music.
        # Scale: A - B - C - D - E - F - G - A
        # Intervals: T - B - T - T - B - T - T (9 - 4 - 9 - 9 - 4 - 9 - 9)
        "offsets": [0, 9, 13, 22, 31, 35, 44, 53],
        "notes": ["Dugah (A)", "Buselik (B)", "Chargah (C)", "Neva (D)", "Huseyni (E)", "Acem (F)", "Gerdaniye (G)", "Muhayyer (A)"]
    },

    "Kurdi_A": {
        # Kurdi maqam is centered on A (Dugah).
        # Scale: A - Bb (Kurdi/B) - C - D - E - F - G - A
        # Note: The Kurdi pitch (B) here can be taken as Bakiye (4 commas) or Kucuk Mucennep (5 commas).
        # Generally, the Kurdi Tetrachord is accepted as: B - T - T (4 - 9 - 9).
        "offsets": [0, 4, 13, 22, 31, 35, 44, 53],
        "notes": ["Dugah (A)", "Kurdi (B b4)", "Chargah (C)", "Neva (D)", "Huseyni (E)", "Acem (F)", "Gerdaniye (G)", "Muhayyer (A)"]
    },

    "Huseyni_A": {
        # Starts very similarly to Ussak but differentiates in the upper register (Evic pitch).
        # While Ussak uses F (Acem), Huseyni uses F# (Evic) when ascending, and F (Acem) when descending.
        # For analysis, using the "ascending" scale (with Evic) is better to show the spectral difference from Ussak.
        # Intervals: K - S - T - T - K - S - T (Huseyni Pentachord + Ussak Tetrachord)
        "offsets": [0, 8, 13, 22, 31, 39, 44, 53],
        "notes": ["Dugah (A)", "Segah (B b1)", "Chargah (C)", "Neva (D)", "Huseyni (E)", "Evic (F #4)", "Gerdaniye (G)", "Muhayyer (A)"]
    },

    "Segah_B": {
        # The tonic is NOT A (Dugah). The tonic is the Segah (B) pitch.
        # The frequency spectrum of this maqam will be completely shifted, making it great test data for analysis.
        # If Dugah(0) is the reference, the Segah pitch is at the 8th comma. The scale starts here.
        # Scale: Segah - Chargah - Neva - Huseyni - Evic - Gerdaniye - Muhayyer - Tiz Segah
        "offsets": [8, 13, 22, 31, 39, 44, 53, 61],
        "notes": ["Segah (B)", "Chargah (C)", "Neva (D)", "Huseyni (E)", "Evic (F#)", "Gerdaniye (G)", "Muhayyer (A)", "Tiz Segah (B)"]
    },
    
    "Saba_A": {
        # Contains one of the most characteristic intervals in maqam music.
        # Interpreted as Saba Tetrachord (K - S - K) or (S - S - K), but the Hicaz pitch (Db/C#) is characteristic.
        # Scale: A - B(Segah) - C(Chargah) - Db(Hicaz) - E(Huseyni) ...
        # The Hicaz pitch (C#) here is approximately 4-5 commas above Chargah (13).
        "offsets": [0, 8, 13, 17, 31, 35, 44, 53], 
        "notes": ["Dugah (A)", "Segah (B)", "Chargah (C)", "Hicaz (Db/C#)", "Huseyni (E)", "Acem (F)", "Gerdaniye (G)", "Muhayyer (A)"]
    },

    "Rast_G": {
        # Rast Maqam is centered on G (Rast pitch). It is 9 commas below A (Dugah).
        # Scale: G(Rast) - A(Dugah) - B(Segah) - C(Chargah) - D(Neva) - E(Huseyni) - F#(Evic) - G(Gerdaniye)
        # Intervals: T - K - S - T - T - K - S (9 - 8 - 5 - 9 - 9 - 8 - 5)
        "offsets": [-9, 0, 8, 13, 22, 31, 39, 44],
        "notes": ["Rast (G)", "Dugah (A)", "Segah (B b5)", "Chargah (C)", "Neva (D)", "Huseyni (E)", "Evic (F #4)", "Gerdaniye (G)"]
    },
    
    "Hicaz_A": {
        # Hicaz Maqam is centered on A (Dugah).
        # Scale: A - B b5 (Dik Kurdi/S) - C #(Hicaz/A) - D - E - F - G - A
        # Intervals: S - A(12) - S - T - B - T - T (Hicaz-Huseyni scale)
        # Note: Hicaz Tetrachord (S-A-S) -> 5 + 12 + 5 = 22 (Perfect Fourth)
        "offsets": [0, 5, 17, 22, 31, 35, 44, 53],
        "notes": ["Dugah (A)", "Dik Kurdi (B b5)", "Hicaz (C #4)", "Neva (D)", "Huseyni (E)", "Acem (F)", "Gerdaniye (G)", "Muhayyer (A)"]
    },

    "Ussak_A": {
        # Ussak Maqam is centered on A (Dugah).
        # Scale: A - B (Segah/K) - C - D - E - F - G - A
        # Intervals: K - S - T - T - B - T - T (Ussak Tetrachord + Buselik Pentachord?? Usually F changes in Ussak progression, but this is the basic scale)
        "offsets": [0, 8, 13, 22, 31, 35, 44, 53],
        "notes": ["Dugah (A)", "Segah (B b1)", "Chargah (C)", "Neva (D)", "Huseyni (E)", "Acem (F)", "Gerdaniye (G)", "Muhayyer (A)"]
    },
    
    "Nihavend_A": {
        # Nihavend is on A (Similar to Buselik scale but the progression/seyir is different, listing the scale here)
        # Scale: A - B (Buselik/T) - C - D - E - F (B) - G - A
        # Closest to Western Minor, but commas differ.
        # Intervals: T - B - T - T - B - T - T
        "offsets": [0, 9, 13, 22, 31, 35, 44, 53],
        "notes": ["Dugah (A)", "Buselik (B)", "Chargah (C)", "Neva (D)", "Huseyni (E)", "Acem (F)", "Gerdaniye (G)", "Muhayyer (A)"]
    }
}

# Processing and Recording 


In [11]:
csv_data = []

for maqam_name, data in maqam_data.items():
    offsets = data["offsets"]
    note_names = data["notes"]
    
    audio_sequence = np.array([], dtype=np.float32)
    silence = np.zeros(int(sr * 0.1)) # Silence/Gap between notes
    
    # Ascending Scale
    for i, offset in enumerate(offsets):
        freq = get_freq(offset)
        tone = generate_tone(freq, 0.6, sr)
        audio_sequence = np.concatenate((audio_sequence, tone, silence))
        
        
        csv_data.append([maqam_name, note_names[i], offset, f"{freq:.4f}"])
        
    # Descending Scale 
    for i in range(len(offsets)-2, -1, -1):
        offset = offsets[i]
        freq = get_freq(offset)
        tone = generate_tone(freq, 0.6, sr)
        audio_sequence = np.concatenate((audio_sequence, tone, silence))

    
    if len(audio_sequence) > 0:
        max_val = np.max(np.abs(audio_sequence))
        if max_val > 0:
            audio_sequence = audio_sequence / max_val * 0.9
            
        filename = f"{maqam_name}_scale.wav"
        wavfile.write(os.path.join(out_dir, filename), sr, (audio_sequence * 32767).astype("int16"))
     

In [12]:

with open(os.path.join(out_dir, "maqam_analysis_data.csv"), "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Maqam", "NoteName", "CommaIndex(Ref_A_0)", "FrequencyHz"])
    writer.writerows(csv_data)

